In [1]:
with open('data/aoforce.out') as f:
    raw = f.read()

In [3]:
from io import StringIO

In [39]:
sample = '''

       mode              61       62       63       64       65       66

     frequency        2944.50  2965.81  2987.14  3011.15  3098.67  3103.94

     symmetry            a        a        a        a        a        a   

        IR               YES      YES      YES      -        YES      YES
|dDIP/dQ|   (a.u.)     0.0044   0.0044   0.0040   0.0051   0.0013   0.0012
intensity (km/mol)      34.90    34.99    28.78    47.10     2.92     2.56
intensity (  %   )      17.49    17.53    14.42    23.60     1.46     1.28
 
       RAMAN             YES      YES      YES      YES      YES      YES

'''

In [5]:
import re

In [15]:
label = 'mode', pattern = '\d+', n = 6, converter = int

match = re.search(r'^\s+mode((\s+\d+){6})\s+$', sample, re.MULTILINE)
[int(mode_id) for mode_id in match.groups(0)[0].split()]

In [23]:
match = re.search(r'^\s+frequency((\s+[\d.]+){6})\s+$', sample, re.MULTILINE)
[float(freq) for freq in match.groups(0)[0].split()]

[2944.5, 2965.81, 2987.14, 3011.15, 3098.67, 3103.94]

In [40]:
match = re.search(r'^\s+IR((\s+(YES)|(-)){6}\s*$)', sample, re.MULTILINE)
match
#[float(freq) for freq in match.groups(0)[0].split()]

In [44]:
from pyparsing import Word, nums, Literal, Or

In [62]:
Integer = Word(nums).setParseAction(lambda x: int(x[0]))
Float = Word(nums+'.')
YesNo = Or(['-', 'YES'])

In [68]:
ir_activity = Literal('IR') + (YesNo * 6)
mode_label = Literal('mode') + (Integer('is') * 3) + (Integer('iis') * 3)

In [77]:
res = ir_activity.scanString(sample)

In [79]:
next(res)

((['IR', 'YES', 'YES', 'YES', '-', 'YES', 'YES'], {}), 236, 301)